In [71]:
import json
import os
import sys

import joblib

# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# Импортируем metrics из utils
from utils import metrics as mtc
from utils import preprocess_functions as ppf

In [72]:
import numpy as np
import pandas as pd

# Готовим дата-сеты (предобработка)

In [73]:
reference = pd.read_csv("../../data/preprocessed/general/reference.csv")
train = pd.read_csv("../../data/splited/train.csv")
valid = pd.read_csv("../../data/splited/valid.csv")

abbr_dict = joblib.load("../../resources/general/abbreviations_dict.joblib")
region_dict = joblib.load("../../resources/general/region_dict.joblib")

In [74]:
reference["processed_name"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.processed_name = reference.processed_name.apply(ppf.replace_numbers_with_text)
reference.processed_name = reference.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
reference.processed_name = reference.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
reference.head(1)

,id,name,region,processed_name
0,69,Кировска,мурманская область,кировска


In [183]:
reference[reference.id == 198]

,id,name,region,processed_name
237,198,СПБ ГБУ ДО СШОР «Академия ФКК»,санкт петербург,государственное бюджетное учреждение дополните...


In [75]:
train["processed_name"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.processed_name = train.processed_name.apply(ppf.replace_numbers_with_text)
train.processed_name = train.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
train["region"] = train.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

train.processed_name = train.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
train.head(1)

,name,school_id,processed_name,region
0,"Москва, ГБУ МАФКК школа ""Мечта""",102,государственное бюджетное учреждение московска...,москва


In [76]:
valid["processed_name"] = valid.name.apply(ppf.simple_preprocess_text).str.lower()
valid.processed_name = valid.processed_name.apply(ppf.replace_numbers_with_text)
valid.processed_name = valid.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
valid["region"] = valid.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

valid.processed_name = valid.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
valid.head(1)

,name,school_id,processed_name,region
0,"Санкт-Петербург, СПб СШОР",198,спортивная школа олимпийского резерва,санкт петербург


# Данные для обучения

In [77]:
merged_train = train.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_train = merged_train[~merged_train.name_reference.isna()]
merged_train = merged_train[~merged_train.processed_name.isna()]
merged_train = merged_train[
    [
        "processed_name",
        "region",
        "processed_name_reference",
        "region_reference",
    ]
]
merged_train["target"] = 1

In [78]:
merged_train.head()

,processed_name,region,processed_name_reference,region_reference,target
0,государственное бюджетное учреждение московска...,москва,мечта,москва,1
1,спортивная школа космос,калужская область,космос,калужская область,1
2,семь округ общество с ограниченной ответственн...,None,династия,санкт петербург,1
3,автономная некоммерческая физкультурно спортив...,None,спартак,санкт петербург,1
4,индивидуальный предприниматель дмитриев,челябинская область,команда дмитриева,челябинская область,1


In [79]:
temp_df = merged_train.copy()
train_classif = merged_train.copy()

for i in range(0, 1, 1):
    temp_df = temp_df.copy()

    # Параметр для сдвига
    shift_amount = 15  # Положительное значение сдвигает вниз, отрицательное - вверх

    # Сдвиг второго столбца
    temp_df["processed_name_reference"] = temp_df["processed_name_reference"].shift(
        shift_amount
    )
    temp_df["region_reference"] = temp_df["region_reference"].shift(shift_amount)
    temp_df["target"] = 0

    # Новый датафрейм с исходным и сдвинутым столбцами
    temp_df = temp_df.dropna()

    train_classif = pd.concat([train_classif, temp_df])

train_classif.reset_index(drop=True, inplace=True)

In [80]:
train_classif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   processed_name            966 non-null    object
 1   region                    844 non-null    object
 2   processed_name_reference  966 non-null    object
 3   region_reference          966 non-null    object
 4   target                    966 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 37.9+ KB


In [81]:
train_classif = train_classif.fillna(" ")

In [82]:
# Объединение текстов для векторизации
train_classif["combined"] = (
    (
        train_classif["processed_name"]
        + " "
        + train_classif["region"]
        + " "
        + train_classif["processed_name_reference"]
        + " "
        + train_classif["region_reference"]
        + " "
    )
    .str.strip()
    .str.replace("  ", " ")
)

In [83]:
# Разделение на признаки и целевую переменную
X = train_classif["combined"]
y = train_classif["target"]

In [84]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# CATBOOST

In [85]:
from catboost import CatBoostClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [86]:
# Создание пайплайна для векторизации, балансировки классов и обучения модели логистической регрессии
pipeline = ImbPipeline(
    [
        ("tfidf", TfidfVectorizer()),
        # ("smote", SMOTE(sampling_strategy="auto", k_neighbors=5, random_state=42)),
        (
            "clf",
            CatBoostClassifier(
                iterations=1000, verbose=100, auto_class_weights="Balanced"
            ),
        ),
    ]
)

In [87]:
# Обучение модели
pipeline.fit(X_train, y_train)

Learning rate set to 0.009225
0:	learn: 0.6891371	total: 7.54ms	remaining: 7.53s


100:	learn: 0.4436481	total: 1.21s	remaining: 10.7s
200:	learn: 0.3460918	total: 2.21s	remaining: 8.77s
300:	learn: 0.2936004	total: 2.88s	remaining: 6.68s
400:	learn: 0.2581021	total: 3.8s	remaining: 5.67s
500:	learn: 0.2317285	total: 4.47s	remaining: 4.46s
600:	learn: 0.2098756	total: 5.07s	remaining: 3.37s
700:	learn: 0.1896430	total: 5.66s	remaining: 2.41s
800:	learn: 0.1761940	total: 6.25s	remaining: 1.55s
900:	learn: 0.1586586	total: 6.9s	remaining: 758ms
999:	learn: 0.1434349	total: 7.47s	remaining: 0us


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x000002385868E870>)])

In [88]:
# Оценка модели
y_pred_test = pipeline.predict(X_test)

In [89]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90        84
           1       0.91      0.95      0.93       110

    accuracy                           0.92       194
   macro avg       0.92      0.91      0.92       194
weighted avg       0.92      0.92      0.92       194



# Данные для валидации

In [90]:
merged_valid = train.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_valid = merged_valid[~merged_valid.name_reference.isna()]
merged_valid = merged_valid[~merged_valid.processed_name.isna()]
merged_valid = merged_valid[
    [
        "processed_name",
        "region",
        "processed_name_reference",
        "region_reference",
    ]
]

merged_valid = merged_valid.fillna(" ")

In [91]:
merged_valid.head()

,processed_name,region,processed_name_reference,region_reference
0,государственное бюджетное учреждение московска...,москва,мечта,москва
1,спортивная школа космос,калужская область,космос,калужская область
2,семь округ общество с ограниченной ответственн...,,династия,санкт петербург
3,автономная некоммерческая физкультурно спортив...,,спартак,санкт петербург
4,индивидуальный предприниматель дмитриев,челябинская область,команда дмитриева,челябинская область


In [92]:
valid_pos_classif = pd.DataFrame()

# Объединение текстов для векторизации
valid_pos_classif["left"] = merged_valid["processed_name"]
valid_pos_classif["left_region"] = merged_valid["region"]
valid_pos_classif["right"] = merged_valid["processed_name_reference"]
valid_pos_classif["right_region"] = merged_valid["region_reference"]

valid_pos_classif = valid_pos_classif.fillna(" ")

valid_pos_classif["combined"] = (
    valid_pos_classif["left"]
    + " "
    + valid_pos_classif["left_region"]
    + " "
    + valid_pos_classif["right"]
    + " "
    + valid_pos_classif["right_region"]
)
valid_pos_classif = valid_pos_classif.reset_index(drop=True)

In [93]:
# Оценка модели
y_pred_valid_pos = pipeline.predict(valid_pos_classif["combined"])

In [94]:
y_pred_valid_pos

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [195]:
# Оценка модели
y_pred_valid_pos_proba = pipeline.predict_proba(valid_pos_classif["combined"])[:, 1]
min(y_pred_valid_pos_proba)


0.36435391072059053

In [95]:
# Качество предсказания класса 1
sum(y_pred_valid_pos) / len(y_pred_valid_pos)

0.9872727272727273

In [117]:
valid_neg_classif = pd.DataFrame()

# Объединение текстов для векторизации
valid_neg_classif["left"] = merged_valid["processed_name"]
valid_neg_classif["left_region"] = merged_valid["region"]
valid_neg_classif["right"] = merged_valid["processed_name_reference"].shift(15)
valid_neg_classif["right_region"] = merged_valid["region_reference"].shift(15)

valid_neg_classif["combined"] = (
    valid_neg_classif["left"]
    + " "
    + valid_neg_classif["left_region"]
    + " "
    + valid_neg_classif["right"]
    + " "
    + valid_neg_classif["right_region"]
)

valid_neg_classif = valid_neg_classif[~valid_neg_classif.combined.isna()]

valid_neg_classif = valid_neg_classif.reset_index(drop=True)

In [118]:
valid_neg_classif

,left,left_region,right,right_region,combined
0,спорт индустрия великий новгород,,мечта,москва,спорт индустрия великий новгород мечта москва
1,темпо,санкт петербург,космос,калужская область,темпо санкт петербург космос калужская область
2,государственное бюджетное учреждение московска...,москва,династия,санкт петербург,государственное бюджетное учреждение московска...
3,государственное автономное учреждение спортив...,рязанская область,спартак,санкт петербург,государственное автономное учреждение спортив...
4,государственное бюджетное учреждение дополните...,москва,команда дмитриева,челябинская область,государственное бюджетное учреждение дополните...
...,...,...,...,...,...
530,государственное бюджетное учреждение дополните...,москва,голубева,костромская область,государственное бюджетное учреждение дополните...
531,муниципальное бюджетное учреждение спортивная ...,калужская область,чемпион,санкт петербург,муниципальное бюджетное учреждение спортивная ...
532,государственное бюджетное учреждение дополните...,москва,красная звезда,санкт петербург,государственное бюджетное учреждение дополните...
533,спортивный клуб спортивно тренировочный центр ...,ставропольский край,спортивный клуб фигурного катания москвиной,санкт петербург,спортивный клуб спортивно тренировочный центр ...


In [119]:
# Оценка модели
y_pred_valid_neg = pipeline.predict(valid_neg_classif["combined"])

In [193]:
# Оценка модели
y_pred_valid_neg_proba = pipeline.predict_proba(valid_neg_classif["combined"])[:, 1]
max(y_pred_valid_neg_proba)


0.9208461037592551

In [120]:
y_pred_valid_neg

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [121]:
# Качество предсказания класса 0
(len(y_pred_valid_neg) - sum(y_pred_valid_neg)) / len(y_pred_valid_neg)

0.8261682242990654

# Изучение остатков

In [122]:
ids_pos_wrong = np.where(y_pred_valid_pos == 0)[0]
len(ids_pos_wrong.tolist())

7

In [123]:
ids_pos_wrong

array([ 74,  85, 153, 246, 340, 372, 411], dtype=int64)

In [124]:
valid_pos_classif.loc[
    ids_pos_wrong.tolist(),
    [
        "left",
        "left_region",
        "right",
        "right_region",
    ],
]

,left,left_region,right,right_region
74,муниципальное автономное учреждение спортивная...,,спортивная школа шесть,мурманская область
85,муниципальное бюджетное учреждение спортивная ...,свердловская область,юность,свердловская область
153,муниципальное бюджетное учреждение всеволжская...,,всеволжская спортивная школа олимпийского резерва,ленинградская область
246,спортивная школа юность,свердловская область,юность,свердловская область
340,муниципальное бюджетное учреждение спортивная ...,,тодес,челябинская область
372,сам область московская академия фигурного ката...,москва,вдохновение,москва
411,,рязанская область,спортивная школа олимпийского резерва шесть,рязанская область


In [125]:
ids_neg_wrong = np.where(y_pred_valid_neg == 1)[0]
len(ids_neg_wrong.tolist())

93

In [126]:
valid_neg_classif.loc[ids_neg_wrong.tolist(), ["left", "right"]].head(10)

,left,right
0,спорт индустрия великий новгород,мечта
9,чувашия бюджетное учреждение спортивная школа ...,детско юношеская спортивная школа восемь
25,автономная некоммерческая организация физкульт...,спортивный клуб фигурного катания москвиной
36,автономная некоммерческая физкультурно спортив...,золотой конек
46,пермь спортивная школа олимпийского резерва ор...,спортивный клуб фигурного катания москвиной
56,общество с ограниченной ответственностью династия,пахомовой
58,общество с ограниченной ответственностью титул,ледокол
61,вологодская обдасть автономное учреждение васи...,орленок
66,спортивная школа центр псковской области ледов...,ледовый дворец спорта
88,государственное бюджетное учреждение дополните...,спортивная школа псковской области ледовые вид...


# Применение модели

In [127]:
reference_id = reference["id"].to_numpy(dtype="int").flatten()
reference_name = reference["processed_name"].to_numpy(dtype="str").flatten()
reference_region = reference["region"].to_numpy(dtype="str").flatten()

In [128]:
joblib.dump(reference_id, "../../resources/exp40/reference_id.joblib")
joblib.dump(reference_name, "../../resources/exp40/reference_name.joblib")
joblib.dump(reference_region, "../../resources/exp40/reference_region.joblib")

['../../resources/exp40/reference_region.joblib']

In [139]:
x_valid = valid["processed_name"].to_numpy(dtype="str").flatten()
y_valid = valid["school_id"].to_numpy(dtype="int").flatten()
region_valid = valid["region"].to_numpy(dtype="str").flatten()

In [170]:
x_valid_process = np.array([f"{xi} {yi}" for xi, yi in zip(x_valid, region_valid)])
reference_process = np.array(
    [f"{xi} {yi}" for xi, yi in zip(reference_name, reference_region)]
)


In [181]:
y_pred_final = []

for i, x_process in enumerate(x_valid_process):
    reference_ids = np.where(reference_region == region_valid[i])[0]
    print(reference_ids)
    if len(reference_ids) != 0:
        x_test_process = np.char.add(x_process + " ", reference_process[reference_ids])
        print(x_test_process)
        print()
    else:
        x_test_process = np.char.add(x_process + " ", reference_process)

    # Оценка модели
    y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

    # Найти индексы, отсортированные по убыванию значений
    sorted_indices = np.argsort(y_pred_test)[::-1]

    # Выбрать первые пять индексов
    top_five_indices = sorted_indices[:5]

    # Получить значения этих элементов
    top_five_values = y_pred_test[top_five_indices]

    # Создать массив с кортежами (id элемента, значение)
    result = np.array(
        [
            (reference_id[index], value)
            for index, value in zip(top_five_indices, top_five_values)
        ],
        dtype=[("index", int), ("value", float)],
    )

    y_pred_final.append(result)


[ 10  30  47  50  55 117 124 133 135 195 198 200 201 202 203 204 205 210
 211 212 215 222 223 224 225 228 229 230 234 235 236 237 238 239 240 241
 245 246 247 248 249 250 252 253 254 255 256 257 258 259 260 261 262 263
 264 265 266 267 268 269 271 286 292 302 303]
['спортивная школа олимпийского резерва санкт петербург красная звезда санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург бриллиантовый лед санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург династия санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург детско юношеская спортивная школа спортивный клуб юбилейный санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург детско юношеская спортивная школа четыре санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург олимпия санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург прибой санкт петербург'
 'спортивная школа олимпийского резерва санкт петербург спортивная

In [176]:
y_pred_final

[array([(76, 0.88688654), (19, 0.88532996), (53, 0.87623295),
        (20, 0.87623295), (14, 0.87623295)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(69, 0.83329304)], dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(81, 0.79508753), (19, 0.77259836), (25, 0.77143274),
        (70, 0.73970549), (69, 0.6682949 )],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(46, 0.89622414), (45, 0.89434056), (51, 0.89405806),
        (52, 0.89301273), (21, 0.89177389)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(75, 0.78712517), ( 8, 0.74813608), (69, 0.74724646),
        (71, 0.7291525 ), (73, 0.62777305)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(124, 0.92302544), (265, 0.91819403), (220, 0.9172808 ),
        (128, 0.91582801), ( 90, 0.91569114)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(77, 0.87293678), (72, 0.68941256), ( 2, 0.62207323),
        (70, 0.46175415), ( 8, 0.44490311)],
       dtype=[('index

In [ ]:
# Преобразовать итоговый список в массив numpy
y_pred_final = np.array(y_pred_final)


In [184]:
x_valid_process = np.array([f"{xi} {yi}" for xi, yi in zip(x_valid, region_valid)])
reference_process = np.array(
    [f"{xi} {yi}" for xi, yi in zip(reference_name, reference_region)]
)


In [185]:
y_pred_final = []

for x_process in x_valid_process:
    x_test_process = np.char.add(x_process + " ", reference_process)
    # print(x_test_process)
    # Оценка модели
    y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

    # Найти индексы, отсортированные по убыванию значений
    sorted_indices = np.argsort(y_pred_test)[::-1]

    # Выбрать первые пять индексов
    top_five_indices = sorted_indices[:5]

    # Получить значения этих элементов
    top_five_values = y_pred_test[top_five_indices]

    # Создать массив с кортежами (id элемента, значение)
    result = np.array(
        [
            (reference_id[index], value)
            for index, value in zip(top_five_indices, top_five_values)
        ],
        dtype=[("index", int), ("value", float)],
    )

    y_pred_final.append(result)

# Преобразовать итоговый список в массив numpy
y_pred_final = np.array(y_pred_final)

In [186]:
y_pred_final

array([[( 220, 0.88688654), (  89, 0.88532996), (1852, 0.87623295),
        ( 288, 0.87623295), (  29, 0.87623295)],
       [( 300, 0.91137728), ( 184, 0.83329304), ( 145, 0.83093268),
        ( 233, 0.82502265), (1838, 0.80548703)],
       [( 282, 0.83604506), ( 104, 0.79508753), ( 294, 0.77259836),
        (1845, 0.77143274), ( 142, 0.76956725)],
       [( 278, 0.89622414), ( 277, 0.89434056), ( 287, 0.89405806),
        ( 286, 0.89301273), ( 105, 0.89177389)],
       [( 300, 0.87195168), (1848, 0.78712517), (1838, 0.74813608),
        (  35, 0.74724646), ( 145, 0.74015098)],
       [( 124, 0.92302544), ( 265, 0.91819403), ( 220, 0.9172808 ),
        ( 128, 0.91582801), (  90, 0.91569114)],
       [( 110, 0.87293678), ( 300, 0.87128065), ( 145, 0.80295038),
        ( 112, 0.79358579), ( 141, 0.7910214 )],
       [(  11, 0.85159124), ( 140, 0.81926672), ( 299, 0.75366469),
        ( 200, 0.72646661), ( 201, 0.70524628)],
       [( 128, 0.87698534), ( 209, 0.85809444), ( 204, 0.8510690

In [187]:
y_pred_final_1 = [k[0][0] for k in y_pred_final]

In [188]:
df = pd.DataFrame(
    [(y_valid[i], y_pred_final_1[i], y_pred_valid_pos[i]) for i in range(0, 181)],
    columns=["y_valid", "y_pred_final_1", "y_pred_valid_pos"],
)
df

,y_valid,y_pred_final_1,y_pred_valid_pos
0,198,220,1
1,184,300,1
2,293,282,1
3,277,278,1
4,35,300,1
...,...,...,...
176,92,145,1
177,167,80,1
178,90,145,1
179,176,176,1


# Результат

In [189]:
metrics = mtc.calculate_metrics(y_valid, y_pred_final, [])
metrics

{'Accuracy@1': 0.353,
 'Accuracy@3': 0.543,
 'Accuracy@5': 0.674,
 'auto_error_rate': 0.647,
 'manual_processing_rate': 0.0,
 'general_error': 0.647}

In [137]:
# Сохраняем словарь в файл JSON
with open("../../evaluations/exp40.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=4)

# Проверка на одном примере

In [ ]:
(
    "Пользователь ввел: спортивная школа олимпийского резерва, санкт петербург.\n    Выбери наиболее подходящий вариант из следующих:\n    1. спортивная школа олимпийского резерва один, санкт петербург\n2. спортивная школа олимпийского резерва колпинского района, санкт петербург\n3. спортивная школа олимпийского резерва один московского района, санкт петербург\n4. государственное бюджетное учреждение дополнительного образования спортивная школа олимпийского резерва академия фигурного катания на коньках, санкт петербург\n5. спортивная школа олимпийского резерва комплексная филиала федеральное автономное учреждение  российская федерация центральный спортивный клуб армии спортивный клуб армии, санкт петербург\n\n    Напиши только номер ответа.",
)


In [199]:
xx = np.array(
    [
        "спортивная школа олимпийского резерва санкт петербург спортивная школа олимпийского резерва один санкт петербург",
        "спортивная школа олимпийского резерва санкт петербург спортивная школа олимпийского резерва колпинского района санкт петербург",
        "спортивная школа олимпийского резерва санкт петербург спортивная школа олимпийского резерва один московского района санкт петербург",
        "спортивная школа олимпийского резерва санкт петербург государственное бюджетное учреждение дополнительного образования спортивная школа олимпийского резерва академия фигурного катания на коньках санкт петербург",
        "спортивная школа олимпийского резерва санкт петербург спортивная школа олимпийского резерва комплексная филиала федеральное автономное учреждение  российская федерация центральный спортивный клуб армии спортивный клуб армии санкт петербург",
    ]
)


In [200]:
yy = pipeline.predict_proba(xx)[:, 1]

In [201]:
yy

array([0.88688654, 0.87336625, 0.86725174, 0.82225761, 0.6516091 ])

In [ ]:
y_pred_final = []

for x_process in x_valid_process:
    x_test_process = np.char.add(x_process + " ", reference_process)
    # print(x_test_process)
    # Оценка модели
    y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

    # Найти индексы, отсортированные по убыванию значений
    sorted_indices = np.argsort(y_pred_test)[::-1]

    # Выбрать первые пять индексов
    top_five_indices = sorted_indices[:5]

    # Получить значения этих элементов
    top_five_values = y_pred_test[top_five_indices]

    # Создать массив с кортежами (id элемента, значение)
    result = np.array(
        [
            (reference_id[index], value)
            for index, value in zip(top_five_indices, top_five_values)
        ],
        dtype=[("index", int), ("value", float)],
    )

    y_pred_final.append(result)

# Преобразовать итоговый список в массив numpy
y_pred_final = np.array(y_pred_final)
